# Procesamiento de Lenguaje Natural (NLP): Análisis (clasificación) de sentimientos
Juan Diego Lugo Sánchez 201913094

## Importacion de Librerias

In [58]:
import numpy as np
import pandas as pd
import re

from joblib import dump, load
from google.colab import files

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')

from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Subir los datos proporcionados (**Solo para Google Colab**)

In [5]:
uploaded = files.upload()

Saving MovieReviews.csv to MovieReviews.csv


## Leer los datos proporcionados

In [40]:
data=pd.read_csv('MovieReviews.csv', sep=',', encoding = 'utf-8')
data.shape

(5000, 3)

## Limpieza de los datos
Para la limpieza de los datos se pasan las letras a minuscula, reemplazan todos los caracteres especiales y los números por espacios, para finalmente quedarnos con las palabras.

Ejemplo:




---


"Es difícil contarle más sobre esta película sin estropearla.Lo disfruté porque no esperaba lo que estaba viendo, sino un drama sexual ordinario, así que ... es un thriller de PSCYHO-sexual, en el que nada es lo que parece.Cuenta con Emmanuelle Seigner, sin extraño al género (y a la desnudez) en la que su esposo, Polanski, la había dirigido.Y un rendimiento espeluznante (dije espeluznante / sí espeluznante) de Toreton (el actor de Bernard Tavernier).Parece que un Pascal Bruckner se encuentra con Roman Polanski (mejor que la luna amarga), como un chabrol que se fue por extravisos o thriller de Clouzot (he visto a alguien que menciona Les diaboliques), pero más cerca de Georges Franju's Les Yeux Sans Visage (ojos sin cara, laPadrino del Dr. Phibes y más).Una gema !Solo me temo que lo hicieran en un remake de Hollywood como lo hicieron con Nighwatch y la desaparición."


---


es difícil contarle más sobre esta película sin estropearla lo disfruté porque no esperaba lo que estaba viendo sino un drama sexual ordinario así que es un thriller de pscyho sexual en el que nada es lo que parece cuenta con emmanuelle seigner sin extraño al género y a la desnudez en la que su esposo polanski la había dirigido y un rendimiento espeluznante dije espeluznante sí espeluznante de toreton el actor de bernard tavernier parece que un pascal bruckner se encuentra con roman polanski mejor que la luna amarga como un chabrol que se fue por extravisos o thriller de clouzot he visto a alguien que menciona les diaboliques pero más cerca de georges franju s les yeux sans visage ojos sin cara lapadrino del dr phibes y más una gema solo me temo que lo hicieran en un remake de hollywood como lo hicieron con nighwatch y la desaparición


---




In [49]:
data.columns
peliculas = data.review_es 
sentimientos = data.sentimiento

processed_peliculas = []
especiales = r'\W'
espacios= r'\s+'

for pelicula in peliculas:
    pelicula_procesada = pelicula.lower()
    pelicula_procesada = re.sub(especiales, ' ', pelicula_procesada)
    pelicula_procesada = re.sub('[0-9]+', ' ', pelicula_procesada)
    pelicula_procesada = re.sub(espacios, ' ', pelicula_procesada, flags=re.I)
    
    processed_peliculas.append(pelicula_procesada)

ndf = pd.DataFrame(list(zip(processed_peliculas, sentimientos)), columns =['review_es', 'sentimiento'])
data = ndf

ndf.to_csv("MovieReviews_clean.csv")

Finalmente se guarda el csv con los datos ya limpios y se hace una pequeña visualización de los datos.

In [50]:
data.sample(5)

,review_es,sentimiento
2933,para empezar me encanta steven seagal el hombr...,1
4871,la parcela es sobre una enfermera femenina lla...,1
1245,he visto esta película al menos veces y avía e...,2
4088,una vez más un clásico de cine ha sido rehacil...,1
4478,sí el personaje de madsen mientras que habla c...,1


Se revisa el tipo de dato y la cantidad de cada uno de los posibles sentimientos

In [51]:
data["sentimiento"].value_counts()

2    2500
1    2500
Name: sentimiento, dtype: int64

In [52]:
data.shape
sent = data

Se hace el cambio de dato categórico a numérico

In [53]:
sent['sentimiento'].replace('negativo', 1, inplace=True)
sent['sentimiento'].replace('positivo', 2, inplace=True)

### Procesamiento NTLK basico

In [54]:
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))
stemmer = SnowballStemmer('spanish')

In [55]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
    
def tokenize(text):
    text = ''.join([c for c in text if c not in non_words])
    tokens =  word_tokenize(text)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

División de los datos, entrenamiento y evaluación del modelo

In [59]:
cv = CountVectorizer(analyzer = 'word', tokenizer = tokenize, lowercase = True, stop_words = spanish_stopwords)
text_counts = cv.fit_transform(sent['review_es'])

X_train, X_test, Y_train, Y_test = train_test_split(text_counts, sent['sentimiento'], test_size=0.20, random_state=5)

MNB = MultinomialNB()
MNB.fit(X_train, Y_train)

y_pred_train = MNB.predict(X_train)
y_pred_test = MNB.predict(X_test)

print('Exactitud sobre entrenamiento: %.2f' % metrics.accuracy_score(Y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % metrics.accuracy_score(Y_test, y_pred_test))

Exactitud sobre entrenamiento: 0.96
Exactitud sobre test: 0.83
